This notebook explores ensemble methods for model creation. We start with a very simple decision tree and then dive into various ensemble methods.

https://scikit-learn.org/stable/modules/ensemble.html


In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier



In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
titanic= pd.read_csv('/content/drive/MyDrive/data_sets/titanic_cleaned.csv') 

In [8]:
titanic.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,missing,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,missing,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,missing,S


In [9]:
titanic.Pclass = titanic.Pclass.astype(str)

In [10]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 714 entries, 0 to 713
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  714 non-null    int64  
 1   Pclass    714 non-null    object 
 2   Sex       714 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     714 non-null    int64  
 5   Parch     714 non-null    int64  
 6   Fare      714 non-null    float64
 7   Cabin     714 non-null    object 
 8   Embarked  714 non-null    object 
dtypes: float64(2), int64(3), object(4)
memory usage: 50.3+ KB


In [11]:
titanic_enc = pd.get_dummies(titanic)

In [12]:
y_target = titanic_enc.pop('Survived')

Below we will explore a variety of different decision tree models. we begin with a simple default param model, then add in grid search, move to random forest, and finally a boosting classifier. It's worth noting that each of these methods have strengths and weaknesses. Ensemble methods tend to perform better than their more simple counterparts, but as you will see below it's not always quite that simple. Just throwing an ensemble method at a problem won't always improve on score, the algorithm must be implemented correctly to take advantage of the classifiers strengths and minimize the weaknesses. 

One easy to see tradeoff is complexity. The basic decision tree runs in a fraction of the time of these more complex methods. Especially when combined with gridsearch the ensembles are building hundreds to thousands more models than even a grid search decision tree. 

Our dataset is a little over 700 rows. Imagine how long these would take on a dataset with tens of thousands or millions of rows. 

# Decision Tree with default params
Not an ensemble

In [13]:
clf = DecisionTreeClassifier().fit(titanic_enc, y_target)
pd.DataFrame(cross_validate(clf,titanic_enc,y_target,scoring=['f1'])).agg('mean')

fit_time      0.006924
score_time    0.003478
test_f1       0.728035
dtype: float64

# Decision Tree Classifier with GridSearchCV
still not ensemble, but clearly more complex than training a single tree. This explores the parameter space far more thoroughly to find the best performing model.

In [14]:
parameters = {'max_depth':list(range(1,20,2))}
clf = GridSearchCV(DecisionTreeClassifier(), parameters,scoring=['f1'],refit=False)
clf = clf.fit(titanic_enc, y_target)

In [15]:
results = clf.cv_results_
pd.DataFrame(results)[['param_max_depth','mean_test_f1']].sort_values('mean_test_f1',ascending=False).head(10)

,param_max_depth,mean_test_f1
4,9,0.742823
7,15,0.739517
6,13,0.737989
1,3,0.730981
5,11,0.730413
3,7,0.729993
2,5,0.725666
8,17,0.725598
9,19,0.724173
0,1,0.713036


# Random Forest (Bagging) Classifier with GridSearchCV

A bagging model in which predictions are averaged. Bagging often performs better with very deep trees which in general would overfit if it weren't for the averaging of predictions

In [30]:
parameters = {'max_depth':list(range(1,20,2)),
              'n_estimators':[1,2,3,4,5,6,7,8,9,10,100,200,300,400,500,600,700,900,1000]}
clf = GridSearchCV(RandomForestClassifier(n_jobs=-1), parameters,scoring=['f1'],refit=False)
clf = clf.fit(titanic_enc, y_target)
results = clf.cv_results_


KeyboardInterrupt: ignored

In [17]:
pd.DataFrame(results)[['param_max_depth','param_n_estimators','mean_test_f1']].sort_values('mean_test_f1',ascending=False).head(10)

,param_max_depth,param_n_estimators,mean_test_f1
44,5,7,0.742461
124,13,100,0.730266
174,19,4,0.726454
74,7,900,0.724055
119,13,6,0.723342
127,13,400,0.722655
75,7,1000,0.721631
110,11,600,0.721620
102,11,8,0.721104
147,15,500,0.720717


In [18]:
list(range(1,20,2))

[1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

# Gradient Boosting Classifier with GridSearchCV

In [19]:


parameters = {'max_depth':list(range(1,20,2)),
              'n_estimators':[1,2,3,4,5,6,7,8,9,10,100,200,300,400,500,600]
              }
clf = GridSearchCV(GradientBoostingClassifier(), parameters,scoring=['f1'],refit=False)
clf = clf.fit(titanic_enc, y_target)
results = clf.cv_results_

In [20]:
pd.DataFrame(results)[['param_n_estimators','param_max_depth','mean_test_f1']].sort_values('mean_test_f1',ascending=False).head(10)

,param_n_estimators,param_max_depth,mean_test_f1
31,600,3,0.755406
30,500,3,0.752764
43,200,5,0.748769
28,300,3,0.745088
57,10,7,0.743702
87,8,11,0.742448
88,9,11,0.742383
55,8,7,0.742290
56,9,7,0.741985
29,400,3,0.741820


# Voting Ensemble

Predicts the class that the majority of classifiers predicted. In this case if we have the three classifiers predict

1 predicts class 0

2 predicts class 1

3 predicts class 0

then the final prediction will be 0



In [21]:
clf1 = SVC(kernel='sigmoid',random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = SVC(kernel='rbf',random_state=1)

eclf = VotingClassifier(
    estimators=[('svc_sig', clf1), ('rf', clf2), ('svc_rbf', clf3)],
    voting='hard')


eclf = eclf.fit(titanic_enc, y_target)
pd.DataFrame(cross_validate(eclf,titanic_enc,y_target,scoring=['f1'])).agg('mean')


fit_time      0.130952
score_time    0.026891
test_f1       0.641739
dtype: float64

# Voting with Grid Search

Here we show that it is possible to combine our use of GridSearch, Cross-Validation and Ensemble methods. While the performance is not as good in this case it is a very robust method that can outperform in some circumstances. 

In [22]:
# dropping the number of grid search params to reduce fit time

# start with a sigmoid SVC
parameters_svc_sig = {'C':list(range(1,10,2))}
clf1 = GridSearchCV(SVC(kernel='sigmoid',random_state=1), parameters_svc_sig,scoring='f1',refit=True)


# Add in a random forest
parameters_dt = {'max_depth':list(range(1,20,2)),
                'n_estimators':[1,2,3,4,5,6,7,8,9,10,100,200]
                 }
clf2 = GridSearchCV(RandomForestClassifier(), parameters_dt,scoring='f1',refit=True)

# and finally a SVC with a radial kernel
parameters_svc_rbf = {'C':list(range(1,10,2))}
clf3 = GridSearchCV(SVC(kernel='rbf',random_state=1), parameters_svc_rbf,scoring='f1',refit=True)

eclf = VotingClassifier(
    estimators=[('svc_sig', clf1), ('rf', clf2), ('svc_rbf', clf3)],
    voting='hard')


eclf = eclf.fit(titanic_enc, y_target)
pd.DataFrame(cross_validate(eclf,titanic_enc,y_target,scoring=['f1'])).agg('mean')

fit_time      31.166238
score_time     0.028050
test_f1        0.658889
dtype: float64

# Stacked Ensemble

Stacking is one of the more complex models we will explore in this class. It combines multiple models each trained using a cross validated dataset into a new single gradient boosted model which is then trained using cross validation again. 

In [23]:
clf1 = SVC(kernel='sigmoid',random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = SVC(kernel='rbf',random_state=1)

In [24]:
estimators = [('svc_sig', clf1),
              ('rf', clf2),
              ('svc_rbf', clf3)]

In [25]:

final_estimator = GradientBoostingClassifier(
    n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1,
    random_state=42)



stacked_ens = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator)

pd.DataFrame(cross_validate(stacked_ens,titanic_enc,y_target,scoring=['f1'])).agg('mean')

fit_time      0.797311
score_time    0.023937
test_f1       0.692399
dtype: float64

# Stacking Ensemble with GridSearchCV

Now we stack but throw in grid search to try and get the best single model for each classifier. 

In [26]:
parameters_svc_sig = {'C':list(range(1,3,1))}
clf1 = GridSearchCV(SVC(kernel='sigmoid',random_state=1), parameters_svc_sig,scoring='f1',refit=True)


# Add in a random forest
parameters_dt = {'max_depth':list(range(1,3,1)),
                'n_estimators':[1,2,3,4,5] # reduce the grid search to make the model fit faster (it won't be as robust though)
                 }
clf2 = GridSearchCV(RandomForestClassifier(), parameters_dt,scoring='f1',refit=True)

# and finally a SVC with a radial kernel
parameters_svc_rbf = {'C':list(range(1,3,1))}
clf3 = GridSearchCV(SVC(kernel='rbf',random_state=1), parameters_svc_rbf,scoring='f1',refit=True)

In [27]:
estimators = [('svc_sig', clf1),
              ('rf', clf2),
              ('svc_rbf', clf3)]

note that max_features = 1. this new feature is actually the prediction from each of the estimators 

In [28]:
# note that the performance here will be underwhelming as combinging grid search with a stacking classifier is a computationally expensive 
# procedure. previous runs of this using the same grid params as models above took over 2 hours to run
# this code is mainly for demonstration purposes therefore and should not be interpreted as a 
# reason to not consinder stacking due to poor performance

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import StackingClassifier


final_estimator = GradientBoostingClassifier(
    n_estimators=25, subsample=0.5, min_samples_leaf=25, max_features=1,
    random_state=42)


stacked_ens = StackingClassifier(
    estimators=estimators,
    final_estimator=final_estimator)

pd.DataFrame(cross_validate(stacked_ens,titanic_enc,y_target,scoring=['f1'])).agg('mean')

fit_time      5.865926
score_time    0.019592
test_f1       0.637845
dtype: float64

In [29]:
!cp "/content/drive/My Drive/Colab Notebooks/4482_ensemble.ipynb" ./

# run the second shell command, jupyter nbconvert --to html "file name of the notebook"
# create html from ipynb

!jupyter nbconvert --to html "4482_ensemble.ipynb"

[NbConvertApp] Converting notebook 4482_ensemble.ipynb to html
[NbConvertApp] Writing 326374 bytes to 4482_ensemble.html
